In [540]:
import requests
import urllib3
import pandas as pd
from datetime import datetime
from datetime import time
from datetime import date

Interesting link: https://towardsdatascience.com/using-the-strava-api-and-pandas-to-explore-your-activity-data-d94901d9bfde
Strava API forum: https://groups.google.com/g/strava-api

In [541]:
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

auth_url = "https://www.strava.com/oauth/token"
payload = {
    'client_id': "75483",
    'client_secret': 'd7d1e5483201559ddf62c677ac12f6cf7847c5ae',
    'refresh_token': '75994cf8182a3b6a956390b5a4da3eba764f1d85',
    'grant_type': "refresh_token",
    'f': 'json',
}

In [542]:
print("Requesting Token...\n")
res = requests.post(auth_url, data=payload, verify=False)
access_token = res.json()['access_token']
print("Access Token = {}\n".format(access_token))

Requesting Token...

Access Token = 1e8359c635f1a6e7757aed4aecbb2463b1d228d2



In [543]:
activites_url = "https://www.strava.com/api/v3/athlete/activities"

dataset=[]
pages_count=1

while pages_count>0:
    header = {'Authorization': 'Bearer ' + access_token}
    param = {'per_page': 200, 'page': pages_count}
    my_dataset=requests.get(activites_url, headers=header, params=param).json()
    if my_dataset==[]:
        break
    elif my_dataset!=[]:
        dataset.append(my_dataset)
        pages_count+=1
dataset = [j for i in dataset for j in i]


In [544]:
activities = pd.json_normalize(dataset)

In [545]:
#Create new dataframe with only columns I care about
cols = ['name', 'upload_id', 'type', 'distance', 'moving_time',   
         'average_speed', 'max_speed','total_elevation_gain',
         'start_date_local'
       ]

activities = activities[cols]

#Break date into start time and date
activities['start_date_local']= [datetime.strptime(i,"%Y-%m-%dT%H:%M:%SZ") for i in activities['start_date_local']]

activities['start_time']=[time(hour=i.hour,minute=i.minute,second=i.second) for i in activities['start_date_local']]
activities['start_date_local'] =[date(year=i.year,month=i.month,day=i.day) for i in activities['start_date_local']]
activities.head(10)

,name,upload_id,type,distance,moving_time,average_speed,max_speed,total_elevation_gain,start_date_local,start_time
0,Carrera de tarde,6869503747,Run,8670.7,2632,3.294,7.142,97.8,2022-01-02,13:43:02
1,Morning Ride,6853686739,Ride,56598.7,8011,7.065,14.076,565.2,2021-12-30,10:30:10
2,Lunch Ride,6849803976,Ride,57733.0,9203,6.273,14.657,860.2,2021-12-29,11:05:09
3,Pic de la Dona + Bastiments,6844920696,Hike,9279.7,13219,0.702,4.366,787.7,2021-12-28,10:35:42
4,Lunch Swim,6844915929,Swim,800.0,836,0.957,0.625,0.0,2021-12-26,12:14:06
5,Morning Ride,6827260236,Ride,34347.8,6825,5.033,12.083,457.4,2021-12-24,10:16:45
6,Afternoon Run,6819310648,Run,8061.9,2497,3.229,3.920,109.8,2021-12-22,13:12:30
7,Evening Swim,6812128544,Swim,1200.0,1419,0.846,1.336,0.0,2021-12-20,18:49:18
8,Lunch Ride,6798101336,Ride,20650.0,4972,4.153,9.545,357.3,2021-12-17,12:30:19
9,Afternoon Run,6785614520,Run,7942.9,2554,3.110,4.383,120.3,2021-12-14,16:50:01


In [546]:
activities['type'].value_counts()

Ride              168
Run               112
Swim               79
Hike               50
Workout             2
BackcountrySki      1
Name: type, dtype: int64

### Get the maps of the routes
https://developers.google.com/maps/documentation/utilities/polylineutility

In [550]:
activities["map.summary_polyline"]


KeyError: 'map.summary_polyline'